In [1]:
# os is used for Ilka's linux :)
#import os
#os.environ["QT_QPA_PLATFORM"] = "xcb"

import cv2
from ultralytics import YOLO


In [8]:
from ultralytics import YOLO

# Load a model
# model = YOLO("yolo11n.yaml")  # build a new model from YAML
# model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)
model = YOLO("yolo11n.yaml").load("yolo11n.pt")  # build from YAML and transfer weights

# Train the model
results = model.train(data="/home/jeski/sem/vc/Visi-n-por-Computador/practica4/datasets/selfmadeYOLO/dataset.yaml", epochs=50, imgsz=640, batch=4)


Transferred 499/499 items from pretrained weights
Ultralytics 8.3.23 🚀 Python-3.10.12 torch-2.5.0+cu124 CPU (Intel Core(TM) i7-8565U 1.80GHz)
engine/trainer: task=detect, mode=train, model=yolo11n.yaml, data=/home/jeski/sem/vc/Visi-n-por-Computador/practica4/datasets/selfmadeYOLO/dataset.yaml, epochs=50, time=None, patience=100, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=yolo11n.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None,

train: Scanning /home/jeski/uni/master/1-semester/vc/Visi-n-por-Computador/practica4/datasets/selfmadeYOLO/labels/train.cache... 42 images, 0 backgrounds, 0 corrupt: 100%|██████████| 42/42 [00:00<?, ?it/s]
val: Scanning /home/jeski/uni/master/1-semester/vc/Visi-n-por-Computador/practica4/datasets/selfmadeYOLO/labels/val.cache... 11 images, 0 backgrounds, 0 corrupt: 100%|██████████| 11/11 [00:00<?, ?it/s]


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G      2.894      10.66      1.735         11        640: 100%|██████████| 11/11 [00:20<00:00,  1.85s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.06it/s]

                   all         11         33          0          0          0          0


...

      50/50         0G       1.74      2.866     0.9457          6        640: 100%|██████████| 11/11 [00:19<00:00,  1.74s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.04it/s]

                   all         11         33      0.524      0.333      0.353      0.169



50 epochs completed in 0.294 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.5MB

Validating runs/detect/train/weights/best.pt...
WARNING ⚠️ validating an untrained model YAML will result in 0 mAP.
Ultralytics 8.3.23 🚀 Python-3.10.12 torch-2.5.0+cu124 CPU (Intel Core(TM) i7-8565U 1.80GHz)
YOLO11n summary (fused): 238 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.10it/s]


                   all         11         33      0.657      0.364      0.363      0.171
Speed: 1.7ms preprocess, 88.9ms inference, 0.0ms loss, 7.9ms postprocess per image
Results saved to runs/detect/train


and now predict

In [8]:
from ultralytics import YOLO
import cv2

# Load your trained YOLO model
model = YOLO("/home/jeski/sem/vc/Visi-n-por-Computador/practica4/datasets/selfmadeYOLO/runs/detect/50-epochs/weights/best.pt")  # Replace with the path to your trained model

# Open the video file
video_path = "/home/jeski/sem/vc/Visi-n-por-Computador/practica4/test-vid-prof.mp4"  # Replace with your video file path
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Set up video writer to save output (optional)
output_path = "/home/jeski/sem/vc/Visi-n-por-Computador/practica4/test-vid-prof-50_epochs-track.mp4"  # Replace with desired output path
fourcc = cv2.VideoWriter_fourcc(*"XVID")
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Process each frame of the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO model on the frame
    results = model(frame)
    
    # Visualize the results on the frame
    annotated_frame = results[0].plot()  # plot() adds bounding boxes and labels to the frame

    # Write the annotated frame to the output video
    out.write(annotated_frame)

# Release resources
cap.release()
out.release()
print(f"Processed video saved to {output_path}")


Error: Could not open video.
Processed video saved to /home/jeski/sem/vc/Visi-n-por-Computador/practica4/test-vid-prof-50_epochs-track.mp4


: 

test with tracking


In [3]:
import cv2
from ultralytics import YOLO

# Load the trained YOLO model
model = YOLO("/home/jeski/sem/vc/Visi-n-por-Computador/practica4/datasets/selfmadeYOLO/runs/detect/50-epochs/weights/best.pt")  # Replace with your model path

# Specify the input and output video paths
video_path = "/home/jeski/sem/vc/Visi-n-por-Computador/practica4/test-vid-prof.mp4"  # Replace with your input video path
output_path = "/home/jeski/sem/vc/Visi-n-por-Computador/practica4/test-vid-prof.mp4"  # Path to save the annotated video

# Open the input video file
cap = cv2.VideoCapture(video_path)

# Get video properties for output video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create VideoWriter object
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (frame_width, frame_height))

# Iterate through each frame in the video for tracking
for results in model.track(source=video_path, stream=True):
    # Get the annotated frame
    annotated_frame = results.plot()  # Adds bounding boxes and labels to the frame

    # Write the annotated frame to the output video
    out.write(annotated_frame)

# Release the resources
cap.release()
out.release()

print(f"Annotated video saved to {output_path}")




WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/2832) /home/jeski/sem/vc/Visi-n-por-Computador/practica4/test-vid-prof.mp4: 384x640 (no detections), 88.5ms
video 1/1 (frame 2/2832) /home/jeski/sem/vc/Visi-n-por-Computador/practica4/test-vid-prof.mp4: 384x640 1 license plate, 72.6ms
video 1/1 (frame 3/2832) /home/jeski/sem/vc/Visi-n-por-Computador/practica4/test-vid-prof.mp4: 384x640 1 license plate, 192.5ms
video 1/1 (frame 4/2832) /home/jeski/sem/vc/Visi-n-por-Computador/practica

KeyboardInterrupt: 

test with image


In [3]:
import cv2
from ultralytics import YOLO

# Load the trained YOLO model
model = YOLO("/home/jeski/sem/vc/Visi-n-por-Computador/practica4/datasets/selfmadeYOLO/runs/detect/train/weights/best.pt")  # Replace with the path to your trained model

# Load the image
image_path = "/home/jeski/sem/vc/Visi-n-por-Computador/practica4/datasets/Our-Images/train/images/1729779190271.jpg"  # Replace with the path to your image
image = cv2.imread(image_path)

# Check if the image was loaded successfully
if image is None:
    print("Error: Could not load image.")
    exit()

# Run YOLO model on the image
results = model(image)

# Visualize the results on the image
annotated_image = results[0].plot()  # plot() adds bounding boxes and labels to the image

# Display the result
#cv2.imshow("YOLO Detection", annotated_image)

# Wait for a key press to close the window
#cv2.waitKey(0)
#cv2.destroyAllWindows()

# Save the result (optional)
output_path = "/home/jeski/sem/vc/Visi-n-por-Computador/practica4/test-1.jpg"  # Specify where to save the result
cv2.imwrite(output_path, annotated_image)
print(f"Annotated image saved to {output_path}")


FileNotFoundError: [Errno 2] No such file or directory: '/home/jeski/sem/vc/Visi-n-por-Computador/practica4/datasets/selfmadeYOLO/runs/detect/train/weights/best.pt'